(page:how-to-plotting)=
# Plotting
This page contains a few ideas for plotting ApRES data using xApRES, starting with simple implementations of xarray plotting functions and moving on to more complex examples. 

In [ ]:
import sys
sys.path.append("../../../xapres") 
import xapres as xa
data = xa.load.load_zarr('gs://ldeo-glaciology/apres/thwaites/continuous/ApRES_LTG/zarr/full.zarr') 
data

## Simple plotting
The xarray documentation includes a comprehensive guide to its plotting routines: https://docs.xarray.dev/en/stable/user-guide/plotting.html.

### Single chirps and profiles

In [ ]:
data.chirp.isel(chirp_num = 50, time = 200).plot(figsize = (20,5), xlim = (0, 1e9));

In [ ]:
data.profile.dB().isel(chirp_num = 50, time = 200).plot(figsize = (20,5), xlim = (0, 2500));

In [ ]:
data.chirp.sel(chirp_num=0, time = slice('2023-05-15','2023-05-30', 4)).plot(row='time', figsize = (10, 10));

### Histograms 


In [ ]:
data.chirp.isel(time = slice(20, 25)).plot.hist();   

```{note}
Usually you would plot a larger subset of the data in a histogram, but `time = slice(20,25)` saves time downloading data in this guide. 
```

A consideration when designing a ApRES survey is avoiding 'clipping' If the returned signal is too high in amplitude multiple components inside the radar can behave in undesirable ways, leading to distortions in the signal or artificial cropping of the chirp at a maximum and minimum value. This is referred to as 'clipping'. Plotting a histogram of the chirp data is useful for choosing attenuator settings to avoid clipping. Usually this is done on the radar unit itself while connected to a laptop in the field, but it can also be useful in post-processing.  

### Time series of profiles

In [ ]:
data.profile_stacked.dB().squeeze().plot(x='time', yincrease = False, figsize = (20,5));

### Time series of velocity/displacement/coherence/phase

In [ ]:
d = data.profile_stacked.displacement_timeseries()

In [ ]:
variable_to_plot = ['coherence','phase','phase_uncertainty','velocity', 'displacement', 'disp_uncertainty']

d[variable_to_plot[4]].plot(figsize = (20,5), y='bin_depth', x='time', yincrease = False);

### Multiple velocity profiles

In [ ]:
d.velocity.sel(time = slice('2023-05-15','2023-06-10', 4)).plot(col = 'time', y = 'bin_depth', yincrease = False, ylim = (1000, 0), xlim = (-10, 10));

### Interactive plot with mutliple profiles

In [ ]:
import hvplot.xarray
import numpy as np

sliced = d.squeeze().sel(time = slice('2023-05-15','2023-06-10'))

sliced.velocity.hvplot(y = "bin_depth", groupby = "time", height = 400, width = 400,  xlim = (-10, 10), flip_yaxis=True)\
    + np.abs(sliced.coherence).hvplot(y = "bin_depth", groupby = "time", height = 400, width = 400,  xlim = (0.5, 1.1), flip_yaxis=True)

### Time series of strain rate

In [ ]:
d.strain_rate.plot(figsize = (20,5));

## More complex plotting

### Progressive stacking

Create an xarray with many stacked profiles, each one created by averaging a different number of profiles.

In [ ]:
da_list=[]
for n in data.chirp_num.values:
    da_list.append(data.isel(time=0, attenuator_setting_pair=0, chirp_num = slice(0,n)).profile.mean(dim='chirp_num').dB())

In [ ]:
import xarray as xr
number_of_profile_stacked = xr.DataArray(np.arange(0, len(data.chirp_num)), dims='number_of_profile_stacked', name='number_of_profile_stacked')
da = xr.concat(da_list, dim = number_of_profile_stacked).rename('profile')

We can take a look at the effect of stacking on the noise floor by plotting out multiple profiles from the xarray we created above, each corresponding to a different number of profiles stacked together. 

In [ ]:
da.isel(number_of_profile_stacked =[1, 4, 20, 99]).plot(xlim = (0, 2500), hue = 'number_of_profile_stacked', figsize = (15, 5));


### Changing parameters and plotting results

`xarray_grid_search.grid_search` is a function currently stored in a github gist that provides a useful way of changing parameters and storing the result in an xarray dataset. If you are running this notebook for the first time you will need to uncomment and run the code in the cell below to download the gist.

In [ ]:
#!getgist jkingslake xarray_grid_search.py

Then we can import the gist and use it to compute a displacement timeseries multiple times, each time using a different set of parameters.

In [ ]:
import xarray_grid_search as xgs
varying_lower_fit_limit = xgs.gridSearch(data.profile_stacked.displacement_timeseries, lower_limit_on_fit=[200, 400, 600])
varying_lower_fit_limit.strain_rate.plot(hue='lower_limit_on_fit');

Unfortunately, `xarray_grid_search.grid_search` does not currently work well when you change parameters that result in changes to the length of the results in any dimension. You can still use the function, but it increases the volume of the data and introduces `nan`s that need to be dealt with before plotting. 

To deal with the common situation when you want to change a parameter that effects the length of the data, you can either apply `xarray_grid_search.grid_search` and interpolate the result to remove the `nan`s (this works, but is inefficient and may be slow for large datasets), or you can do things a bit more manually. 

Here's an example where we interpolate over the `nan`s. 

In [ ]:
varying_bin_size = xgs.gridSearch(data.profile_stacked.displacement_timeseries, bin_size=[10, 20, 40, 50])
varying_bin_size

Plotting variables that only have dimensions that are not the dimensions that changed in length is straightforward:

In [ ]:
varying_bin_size.strain_rate.hvplot( x = 'time')

However, to plot any of the 2D arrays we need to interpolate over the `nan`s using `interpolate_na`. To do this we must first rechunk to have only one chunk in the `bin_depth` dimension:

In [ ]:
varying_bin_size_interpolated = varying_bin_size.chunk(dict(bin_depth=-1)).interpolate_na(dim='bin_depth')
varying_bin_size_interpolated.sel(time = slice('2023-05-15','2023-06-10') ).velocity.squeeze().hvplot(x='time', y = 'bin_depth')

The more manual option avoids creating and interpolating over many `nan`s, so might be quicker in most cases.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = fig, axs = plt.subplots(2, 2, figsize=(10, 8))

data.profile_stacked.displacement_timeseries(lower_limit_on_fit=800, offset = 1).velocity.plot(ax = ax[0,0], x='time', yincrease = False, vmin=-15, vmax = 15)
data.profile_stacked.displacement_timeseries(lower_limit_on_fit=800, offset = 2).velocity.plot(ax = ax[0,1], x='time', yincrease = False, vmin=-15, vmax = 15)
data.profile_stacked.displacement_timeseries(lower_limit_on_fit=800, offset = 3).velocity.plot(ax = ax[1,0], x='time', yincrease = False, vmin=-15, vmax = 15)
data.profile_stacked.displacement_timeseries(lower_limit_on_fit=800, offset = 4).velocity.plot(ax = ax[1,1], x='time', yincrease = False, vmin=-15, vmax = 15)
plt.tight_layout()

## Summary
- Above we have provided various ideas for exploratory plotting of ApRES data once it has been loaded into an xarray dataset.
- xarray's plotting tools make simple plots easy. 
- hvplot is a package that creates interactive plots that are useful for exploring multi-dimensional data. 
- `xarray_grid_search.grid_search` provides a way to run a computation multiple times while changing parameters and store the results, however it does not work well when the length of the data changes. 